# PYNQ Car - Application Notebook

This reference design illustrates how to control the car using Jupyter Notebooks and Python
![Pynq-Car](./imgs/pynq-car.png)

# Drive the car by Human

## Import libraries

In [3]:
#from PIL import Image
import numpy as np
#from pynq import Xlnk
#from pynq import Overlay
import time
import ipywidgets as widgets
from IPython.display import display
from car_control import* 

## Download the Image Processing IP bitstream

In [5]:
pynq_car = Overlay("./Pynq-Car.bit")
timer0 = pynq_car.axi_timer_0
timer1 = pynq_car.axi_timer_1

In [6]:
leds = pynq_car.axi_gpio_1
buttons = pynq_car.axi_gpio_0

## Set the leds in Pynq, you should see the lights change after runing the following code

In [7]:
leds.write(0x04,0x00)
leds.write(0x00,0x0A)

In [15]:
#buttons.write(0x04,0x0F)
#button_status= buttons.read(0x00)
#button_status

0

# Speed Control Example

In [ ]:
#   speed         -100         0           100
# paramenter     90000       140000      180000

## Set the speed zero value and range

In [ ]:
Speed_Zero = 140000.0
Speed_Range = 40000.0

## Try moving forward

In [83]:
pwm(timer0,500000, Speed_Zero)

In [84]:
pwm_start(timer0)

In [88]:
#
pwm(timer0,500000, Speed_Zero+Speed_Range*0.5)

# Steering Engine Control Example

In [52]:
# angle       -60°           0°          60°
#6.07V:  50000  0.5ms, 100000 1.5ms, 150000 1.5ms
#5.00V:  100000 1.0ms, 150000 1.5ms, 200000 2.0ms
#3.30V:  151515 1.5ms, 227272 2.2ms, 303030 3.3ms

## Set the steer zero value and range

In [ ]:
Steer_Zero = 100000.0
Steer_Range = 50000.0

## Try turning left

In [10]:
pwm(timer1,2000000, Steer_Zero)

In [11]:
pwm_start(timer1)

In [12]:
#
pwm(timer1,2000000, Steer_Zero - Steer_Range * 0.5)

# Use the following sliders to control the car

In [3]:
slider1 = widgets.IntSlider(min=-5,max=5,step=1,description='Steer：',value=0)
slider2 = widgets.IntSlider(min=-5,max=5,step=1,description='Speed：',value=0)
display(slider1)
display(slider2)
def steer_handler(attrs):
    pwm(timer1,2000000,Steer_Range*attrs.new/5.0 + Steer_Zero)
    #print(attrs.new)
def speed_handler(attrs):
    pwm(timer0,500000,Speed_Range*attrs.new/5.0 + Speed_Zero)
    #print(attrs.new)
slider1.observe(steer_handler,names='value')
slider2.observe(speed_handler,names='value')

IntSlider(value=0, description='Steer：', max=5, min=-5)

IntSlider(value=0, description='Speed：', max=5, min=-5)

# Drive the car by AI

## What we omit here?
Here we provide a processed .elf which can be run in DPU. If you want to build one, please refer to L2 part.
The whole process should be:
1. Collect training data: control the car using keyboard
2. Train a model
3. Use DNNDK to optimize the model
4. Run the optimized model in DPU
This time we will jump to step 4, the optimized model is provided in `Car/model`.

## End-to-End model
This model inputs images and outputs speed & steer values as control signals.
![AI model](./imgs/ns.png)

## Control structure:Producer & Consumer
![Producer_Consumer](./imgs/p_c.png)

## Load the dpu.ko and compile
Now please open a new terminal and follow the commands below to run the car autonomously.

```shell
cd Car
su
insmod /home/xilinx/dpu.ko
xauth merge /home/xilinx/.Xauthority
make run
```

## Now let's rock
Usage of this exe: ./car c/n/s(n means just use ml, c means use ml & cv, s means use ml and build web server) 0.5(run speed, this time it's useless since the model outputs throttle value directly)

```shell
./build/run s # if the web server building goes wrong, use n instead
```

## You can see the output values through http connection if the web server is up

In [13]:
import requests
import time

# Define two bar
steer_show = widgets.FloatProgress(
    value=0,
    min=-1.,
    max=1.0,
    step=0.1,
    description='Steer:',
    bar_style='info',
    orientation='horizontal'
)
speed_show = widgets.FloatProgress(
    value=0,
    min=-1,
    max=1.0,
    step=0.1,
    description='Speed:',
    bar_style='info',
    orientation='horizontal'
)
display(steer_show, speed_show)




FloatProgress(value=0.0, bar_style='info', description='Steer:', max=1.0, min=-1.0)

FloatProgress(value=0.0, bar_style='info', description='Speed:', max=1.0, min=-1.0)

In [10]:
while True:
    r = requests.get('http://localhost:9090/st')
    steer_show.value = float(r.json()['steer'])
    speed_show.value = float(r.json()['throttle'])
    #print(r.json())
    time.sleep(1)